## Run this only to use in Google Colab, skip otherwise.

In [ ]:
colab = True

# Mount drive ref: https://colab.research.google.com/notebooks/io.ipynb
from google.colab import drive
drive.mount('/content/gdrive')

# Test drive mount
with open('/content/gdrive/My Drive/Colab/cloudchamber/retinanetAllFrames.csv') as f:
    tmp = f.read()
print(tmp.splitlines()[0])

# Convert local path to google drive path
def convertPath(path, 
                replaceFrom = '/home/raymondkwok/data/cloudchamber/', 
                replaceWith = '/content/gdrive/My Drive/Colab/cloudchamber/'):
    return path.replace(replaceFrom, replaceWith)


## Configurations

In [8]:
# if the model has not converted into a inference model, uncomment the line for conversion in "Load modules and models"
model_path = '/home/raymondkwok/git/ml/keras-retinanet/inference_models/resnet50_csv_06.h5'
testFramesPath = '/home/raymondkwok/data/cloudchamber/retinanetAllFrames.csv'
saveFramesDir = '/home/raymondkwok/data/cloudchamber/detect'
progressFile = '/home/raymondkwok/data/cloudchamber/progress'

# labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
labels_to_names = {0: 'alpha', 1: 'beta', 2:'muon'}

# define boundary colors
colorLabelled = (255, 0, 0) #Blue
colorDetected = (0, 255, 0) #Green
colorOverlapped = (0, 0, 255) #Red

if 'colab' in globals():
    testFramesPath = convertPath(testFramesPath)
    saveFramesDir = convertPath(saveFramesDir)
    progressFile = convertPath(progressFile)
    model_path = convertPath(model_path,
                            '/home/raymondkwok/git/ml/keras-retinanet/inference_models/',
                            '/content/gdrive/My Drive/Colab/model_retinanet/cloudchamber/')
    
    try:
        import keras_retinanet
    except:
        !pip install git+https://github.com/fizyr/keras-retinanet.git

[Errno 17] File exists: '/home/raymondkwok/data/cloudchamber/detect'


## Load modules and models

In [9]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import keras
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import matplotlib.pyplot as plt
import cv2
import os
import json
import numpy as np
import time
import pandas as pd
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

def draw_caption2(image, box, caption, loc):
    b = np.array(box).astype(int)
    if loc == 'labelled':
        corr, fontSize, color = (b[0], b[1] - 10), 3, colorLabelled
    elif loc == 'detected':
        corr, fontSize, color = (b[2], b[3] + 10), 3, colorDetected
    elif loc == 'overlapped':
        corr, fontSize, color = (b[0], b[3] + 30), 2, colorOverlapped
    else:
        return
    cv2.putText(image, caption, corr, cv2.FONT_HERSHEY_PLAIN, fontSize, color, 2)
    cv2.putText(image, caption, corr, cv2.FONT_HERSHEY_PLAIN, fontSize, color, 2)

def iou(box1, box2):
    xI1 = max(box1[0], box2[0])
    yI1 = max(box1[1], box2[1])
    xI2 = min(box1[2], box2[2])
    yI2 = min(box1[3], box2[3])

    aI  = max(0,(xI2 - xI1 + 1)) * max(0,(yI2 - yI1 + 1))
    aB1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    aB2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return aI / (aB1+aB2-aI)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

model = models.load_model(model_path, backbone_name='resnet50')

#if the model is not converted to an inference model, use the line below
#model = models.convert_model(model)
#print(model.summary())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# make folders if not existed
try:
    os.makedirs(saveFramesDir)
    tmp, _ = os.path.split(progressFile)
    os.makedirs(tmp)
except Exception as e:
    print(e)
    
#Initialize confusion matrix
labels = ['null'] + [v for k,v in labels_to_names.items()]
stat = pd.DataFrame({l:[0]*len(labels) for l in labels})
stat.index = labels

#Read test frames
tracksImage = pd.read_csv(testFramesPath, header=None, names=['path', 'minx', 'miny', 'maxx', 'maxy', 'label'])
tracksImage['predict'] = pd.Series(['null']*len(tracksImage), index=tracksImage.index)

cnt = 0
group = tracksImage.groupby('path')
groupSize = group.size()

for path, df in group:
    if 'colab' in globals():
        path = convertPath(path)
    image = read_image_bgr(path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    
    # drop low score boxes, and drop overlapped boxes
    isOverlaps = np.array([False]*len(scores[0]))
    for index1, (box1, score1, label1) in enumerate(zip(boxes[0], scores[0], labels[0])):
        if score1 < 0.5:
            break
        for index2, (box2, score2, label2) in enumerate(zip(boxes[0], scores[0], labels[0])):
            if index2 > index1 and iou(box1, box2) >= 0.5:
                isOverlaps[index2] = True
    
    # correct for image scale
    boxes /= scale
    
    # draw all labelled box
    for index, row in df.iterrows():
        try:
            b = [int(n) for n in [row['minx'], row['miny'], row['maxx'], row['maxy']]]
            draw_box(draw, b, color=colorLabelled)
            draw_caption2(draw, b, row['label'], 'labelled')
        except:
            pass
        if row['label'] == '' or type(row['label']) is not str:
            df['label'][index] = 'null'
    
    newRows = [] # store track that is not labelled
    
    # visualize detections
    for box, score, label, isOverlap in zip(boxes[0], scores[0], labels[0], isOverlaps):
        # scores are sorted so we can break
        if score < 0.5:
            break

        #color = label_color(label)
        if isOverlap:
            color = colorOverlapped
            fontSize = 2
        else:
            color = colorDetected
            fontSize = 3
            
        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption2(draw, b, caption, 'overlapped' if isOverlap else 'detected')

        if isOverlap:
            continue
            
        boxFound = False
        predictedLabel = labels_to_names[label]
        
        # determine if the detected track is a labelled track
        for index, row in df.iterrows():
            try:
                if all([abs(int(label)-int(predict)) < 50 
                        for label, predict in zip([row['minx'], row['miny'], row['maxx'], row['maxy']], box)]):
                    df['predict'][index] = predictedLabel
                    boxFound = True
            except:
                pass
        if not boxFound:
            newRows.append([path, '', '', '', '', 'null', predictedLabel])
        
    df = df.append(pd.DataFrame(newRows, columns = ['path', 'minx', 'miny', 'maxx', 'maxy', 'label', 'predict']))
    
    # add prefix to file name for missed track and new discovered track
    folder, file = os.path.split(path)
    prefix1 = ''
    prefix2 = ''
    for index, row in df.iterrows():
        stat[row['predict']][row['label']] += 1
        if row['predict'] == 'null' and row['label'] == 'null':
            continue
        if row['predict'] == 'null':
            prefix1 = 'P_'
        if row['label'] == 'null':
            prefix2 = 'L_'

    print(path)
    progress = {'path':path, 'stat': stat.to_dict()}
    cnt += 1
    if cnt%100 is 2:
        with open(progressFile, 'w') as f:
            f.write(json.dumps(progress))
            print(cnt, '/', groupSize)
            print(stat)
    
    paths = path.split('/')
    file = paths[-2] + '_' + paths[-1]
    cv2.imwrite(os.path.join(saveFramesDir, prefix1 + prefix2 + file), draw)
print('Done!')

processing time:  10.59039044380188


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


       null  alpha  beta  muon
null      0      0     0     0
alpha     0      1     0     0
beta      0      0     0     0
muon      0      0     0     0
/home/raymondkwok/data/cloudchamber/V_20190114_133152_OC0_deBkg/V_20190114_133152_OC0_deBkg_00019.png


KeyboardInterrupt: 